<a href="https://colab.research.google.com/github/mukkatharun/clustering_techniques/blob/main/clustering_techniques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import xgboost as xgb
import numpy as np